<a href="https://colab.research.google.com/github/Mfths/2025_PBO_IFA/blob/main/PENUGASAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===== 1. Upload file =====
from google.colab import files
uploaded = files.upload()  # pilih file gambar (upload "3.jpeg")

# ===== 2. Utilitas Preprocess agar mirip MNIST (28x28, putih-di-atas-hitam) =====
import numpy as np
from PIL import Image, ImageOps

def preprocess_to_mnist_28x28(img_pil):
    """
    Langkah preprocessing:
    - Konversi ke grayscale
    - Auto-contrast
    - (Opsional) invert bila latar terang (agar digit jadi putih, latar jadi gelap seperti MNIST)
    - Crop ke bounding box digit
    - Resize mempertahankan rasio ke (20x20), lalu pad ke (28x28)
    - Normalisasi ke [0,1] dan tambah axis channel
    """
    # 1. Grayscale + autocontrast
    img = img_pil.convert('L')
    img = ImageOps.autocontrast(img)

    arr = np.array(img).astype(np.uint8)

    # 2. Jika rata-rata terang (kertas putih), invert supaya digit jadi putih di atas latar gelap
    if arr.mean() > 127:
        img = ImageOps.invert(img)
        arr = np.array(img)

    # 3. Binarisasi ringan untuk cari bounding box digit
    thr = np.mean(arr) * 0.8  # ambang adaptif sederhana
    mask = arr > thr
    if mask.any():
        ys, xs = np.where(mask)
        y0, y1 = ys.min(), ys.max()
        x0, x1 = xs.min(), xs.max()
        img = img.crop((x0, y0, x1 + 1, y1 + 1))

    # 4. Resize ke 20x20 dengan aspect ratio
    img.thumbnail((20, 20), Image.Resampling.LANCZOS)
    w, h = img.size

    # 5. Pad ke 28x28 dan center
    canvas = Image.new('L', (28, 28), color=0)
    canvas.paste(img, ((28 - w) // 2, (28 - h) // 2))

    # 6. Normalisasi ke [0,1]
    arr = np.array(canvas).astype('float32') / 255.0

    # 7. Tambah channel dim (28,28,1)
    arr = arr[..., None]

    return canvas, arr

from PIL import Image

img = Image.open("3.jpeg")
canvas, arr = preprocess_to_mnist_28x28(img)

# Tampilkan hasil preprocessing
canvas.show()
print("Shape array siap:", arr.shape)


Saving 3.jpeg to 3.jpeg
Shape array siap: (28, 28, 1)


In [3]:
from tensorflow.keras.datasets import mnist
import numpy as np

# 1. Load dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape :", x_test.shape, y_test.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (60000, 28, 28) (60000,)
Test shape : (10000, 28, 28) (10000,)


In [4]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# Flatten
x_train_flat = x_train.reshape(len(x_train), -1) / 255.0
x_test_flat = x_test.reshape(len(x_test), -1) / 255.0

# SVM
clf = svm.SVC(kernel='rbf', gamma='scale')
clf.fit(x_train_flat[:5000], y_train[:5000]) # gunakan subset karena SVM berat
y_pred = clf.predict(x_test_flat)

print("Akurasi:", accuracy_score(y_test, y_pred))

Akurasi: 0.9513


In [5]:
from sklearn.metrics import accuracy_score
import numpy as np
from PIL import Image

results = []

for fname in uploaded.keys():
    # 1. Baca gambar
    img_pil = Image.open(fname)

    # 2. Preprocess ke format MNIST 28x28
    _, x = preprocess_to_mnist_28x28(img_pil)   # x: (28,28,1) float [0,1]

    # 3. Flatten jadi (1, 784) untuk SVM
    x_flat = x.reshape(1, -1)

    # 4. Prediksi dengan model SVM
    pred = int(clf.predict(x_flat)[0])

    # 5. Coba ambil probabilitas prediksi (jika model mendukung)
    conf = None
    try:
        if hasattr(clf, "predict_proba"):
            conf = float(np.max(clf.predict_proba(x_flat)))
    except Exception:
        pass

    # 6. Simpan hasil
    results.append((fname, pred, conf))

    # 7. Print hasil prediksi
    if conf is not None:
        print(f"{fname} -> Prediksi SVM: {pred} (p≈{conf:.2f})")
    else:
        print(f"{fname} -> Prediksi SVM: {pred}")


3.jpeg -> Prediksi SVM: 5
